# Counting Rubik's Snake Loops

Loop is a shape where tail and head are adjacent.

Let's define four numbers for $n$-wedge Rubik's Snake:

* L1(n) - number of formulas (i.e. strings of characters 0,1,2,3 of length $n-1$) that describe shape that are loops.
* L2(n) - number of formulas, deduplicated "up to reversal" that describe loop.
* L3(n) - number of loops (described by n-character formulas) up to cyclic shifts.
* L4(n) - number of loops up to cyclic shifts and reversals.

L4 is the most meaningful number. It is the number of different loops if we look at loops as sets of wedges (ignoring where the tail-head connection is), and consider two loops the same if they can be matched by translation and rotation.

All these values are zeros for odd $n$, and for n=2. So we will be computing values for n=4,6,8,10...

Below are values for n from 4 to 22 that I got.

In [1]:
L1_GOLDEN = [1,8,16,280,2229,20720,226000,2293422,24965960,275633094]
L2_GOLDEN = [1,5,10,145,1129,10405,113113,1147142,12484285,137821030]
L3_GOLDEN = [1,3,3,31,196,1509,14191,127681,1248963,12531157]
L4_GOLDEN = [1,2,3,18,112,777,7198,64056,625584,6267820]

These are all normalized loops (in sense of definition L4) for n=4,6,8,10.

In [2]:
L4_LOOPS = {
  4: {'2222'},
  6: {'131313', '123123'},
  8: {'01230321', '00220022', '12321232'},
  10: {'0131201312', '0021331132', '0021013032', '0013211022', '0231303132', 
       '0121323022', '0230320323', '0113133022', '0012300123', '0113203312', 
       '0123023212', '0131202131', '0013100313', '0021330313', '0120121021', 
       '0013101132', '0022033213', '0231302313'}
}

Code computing the loops.

We enumerate all loops using meet-in the middle, then do deduplication.

In [12]:
import numba
import numpy as np
import time
import functools

from rubiks_snake import RubiksSnakeCounter, _push_wedge,_pop_wedge,_push_next_wedge_if_can, \
    _prepare_arena, _add_wedges_from_formula_while_can, _pop_n_wedges, \
    INIT_WEDGE,BOX_SIZE,FACE_IDS_TO_WEDGE_ID,DY


# Efficient formula encoding.
def encode_formula_as_int(s):
  assert all(48<=ord(c)<=51 for c in s)
  n = len(s)
  return sum((ord(s[i])-48)<<(2*(n-1-i)) for i in range(len(s)))

def decode_formula(code, length):
  return ''.join(str((code>>(2*i))%4) for i in range(length))[::-1]
 
@numba.jit("i8(i8,i8)", inline="always")
def reverse_encoded_formula(code, length):
  ans = 0
  for i in range(length):
    ans = (ans<<2) + (code>>(2*i))%4
  return ans
  
@numba.jit("i8(i8,i8)", inline="always")  
def min_cyclic_shift(code, length):
  ans = code
  l = 2*(length-1)
  for i in range(length - 1):
    code = (code>>2) + ((code&3)<<l)
    if code<ans: ans=code
  return ans

@numba.jit("i8(i8,i8,i8)", inline="always")
def concat_encoded_formulas(code1, code2, length2):
  return (code1<<(2*length2)) + code2

# Test for formula encoding.
for n in range(1,7):
  for code in range(4**n):
    formula = decode_formula(code, n)
    assert len(formula)==n
    assert encode_formula_as_int(formula) == code
    assert decode_formula(reverse_encoded_formula(code,n),n) == formula[::-1]
    min_shift = decode_formula(min_cyclic_shift(code,n), n)
    expected_min_shift = min(formula[i:] + formula[:i] for i in range(n))
    assert min_shift == expected_min_shift
    
    


# Enumerates all shapes.
@numba.jit("(i8[:],i8[:],i8,i8[:],i8[:])")
def _enumerate_shapes_rec(wedges, cubes, cur_formula, formulas, last_wedges):
    last_wedge_index = wedges[0]
    last_wedge = wedges[wedges[0]]
    if wedges[0] == 1:
      next_pos = formulas[0]
      formulas[formulas[0]] = cur_formula
      last_wedges[formulas[0]] = last_wedge
      formulas[0]+=1
      return

    for rot in range(4):
      if _push_next_wedge_if_can(rot, wedges, cubes):
        _enumerate_shapes_rec(wedges, cubes, (cur_formula<<2)+rot, formulas, last_wedges)
        _pop_wedge(wedges, cubes)
        
# Enumerates shapes of length n, their formulas have length n-1.
def enumerate_shapes(n, first_wedge_faces=(0,3)):  
  assert n<=20
  wedges, cubes = _prepare_arena(n, FACE_IDS_TO_WEDGE_ID[first_wedge_faces])  

  num_shapes = RubiksSnakeCounter.S[n]
  formulas = np.zeros(num_shapes+1, dtype=np.int64) 
  last_wedges = np.zeros_like(formulas)
  formulas[0] = 1
  
  _enumerate_shapes_rec(wedges, cubes, 0, formulas, last_wedges)
  
  assert formulas[0] == 1 + num_shapes
  return formulas[1:], last_wedges[1:]


@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L2(f, n):
  f_without_last = f//4
  return min(f_without_last, reverse_encoded_formula(f_without_last, n-1))

@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L3(f, n):
  return min_cyclic_shift(f, n)
        
@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L4(f, n):
  rev = reverse_encoded_formula(f, n)
  return min(min_cyclic_shift(f, n), min_cyclic_shift(rev, n))

@functools.cache
def _count_loops_helper(mode):
  @numba.jit("i8(i8,i8[:],i8[:],i8[:],i8[:])")
  def f(n, formulas_1, last_wedges_1, formulas_2, last_wedges_2):
    half_shapes_count = len(formulas_1)
    assert len(formulas_2) == half_shapes_count
    m = n // 2

    # Manual map to store mapping wedge_id -> List[formula]. 
    linked_list_2 = np.full((half_shapes_count,), -1)
    wedge_id_to_formula_pos_2 = np.full(((BOX_SIZE**3)<<6,), -1)

    for i in range(half_shapes_count):
      last_wedge = last_wedges_2[i]
      if wedge_id_to_formula_pos_2[last_wedge] != -1:
        linked_list_2[i] = wedge_id_to_formula_pos_2[last_wedge] 
      wedge_id_to_formula_pos_2[last_wedge]=i  

    wedges, cubes = _prepare_arena(n, INIT_WEDGE)
    loop_formulas_count = 0 
    normalized_loops = set()
    normalized_loops.add(0)
    normalized_loops.remove(0)

    for i in range(half_shapes_count):
      lookup_key = last_wedges_1[i] ^ 16
      ll_pos = wedge_id_to_formula_pos_2[lookup_key]
      if ll_pos == -1:
        continue
      formula1 = formulas_1[i]
      assert _add_wedges_from_formula_while_can(formula1, m, wedges, cubes) == m
      while ll_pos != -1:
        formula2 = formulas_2[ll_pos]
        ll_pos = linked_list_2[ll_pos]
        formula2_rev = reverse_encoded_formula(formula2, m)      
        added_from_f2 = _add_wedges_from_formula_while_can(formula2_rev, m, wedges, cubes) 
        #assert added_from_f2 <= m-1
        if added_from_f2 == m-1:
          #assert wedges[wedges[0]]>>6 == CENTER_COORD - DY
          if mode == 1:
            loop_formulas_count += 1
          else:
            loop_formula = concat_encoded_formulas(formula1, formula2_rev, m)
            if mode==2:
              norm_formula = normalize_encoded_loop_L2(loop_formula, n)
            elif mode == 3:
              norm_formula = normalize_encoded_loop_L3(loop_formula, n)
            elif mode == 4:
              norm_formula = normalize_encoded_loop_L4(loop_formula, n)   
            normalized_loops.add(norm_formula)
        _pop_n_wedges(added_from_f2, wedges, cubes)    
      _pop_n_wedges(m, wedges, cubes)
      assert wedges[0] == n

    if mode == 1:
      return loop_formulas_count
    else:
      return len(normalized_loops)
  return f
  
# Enumerates all formulas correspdning to loops, no dedup whatsoever.
def count_loops(n, mode=1):
  assert 1<=mode<=4
  if n<4 or n%2==1:
    return 0
  formulas_1, last_wedges_1 = enumerate_shapes(n//2+1, first_wedge_faces=(0,3))
  formulas_2, last_wedges_2 = enumerate_shapes(n//2+1, first_wedge_faces=(3,0))
  return _count_loops_helper(mode)(n, formulas_1, last_wedges_1, formulas_2, last_wedges_2)

In [13]:
for n in range(4,21,2):
  t0=time.time()
  L1,L2,L3,L4 = (count_loops(n, mode=i) for i in range(1,5))
  print("n=%d, L1=%d, L2=%d, L3=%d, L4=%d, time=%fs" % (n,L1,L2,L3,L4, time.time()-t0), flush=True)
  assert L1 == L1_GOLDEN[n//2-2]
  assert L2 == L2_GOLDEN[n//2-2]
  assert L3 == L3_GOLDEN[n//2-2]
  assert L4 == L4_GOLDEN[n//2-2] 

n=4, L1=1, L2=1, L3=1, L4=1, time=16.541888s
n=6, L1=8, L2=5, L3=3, L4=2, time=0.001764s
n=8, L1=16, L2=10, L3=3, L4=3, time=0.001928s
n=10, L1=280, L2=145, L3=31, L4=18, time=0.002371s
n=12, L1=2229, L2=1129, L3=196, L4=112, time=0.003814s
n=14, L1=20720, L2=10405, L3=1509, L4=777, time=0.013397s
n=16, L1=226000, L2=113113, L3=14191, L4=7198, time=0.121573s
n=18, L1=2293422, L2=1147142, L3=127681, L4=64056, time=1.506414s
n=20, L1=24965960, L2=12484285, L3=1248963, L4=625584, time=25.235561s


In [15]:
t0=time.time()
print(count_loops(18,mode=1), time.time()-t0)

2293422 0.25253844261169434


In [16]:
t0=time.time()
print(count_loops(20,mode=1), time.time()-t0)

24965960 3.1920886039733887


In [17]:
t0=time.time()
print(count_loops(22,mode=1), time.time()-t0)

275633094 47.73346781730652


In [18]:
t0=time.time()
print(count_loops(24,mode=1), time.time()-t0)

3069890660 958.3998811244965


In [ ]:
t0=time.time()
print(count_loops(24,mode=4), time.time()-t0)